<a href="https://colab.research.google.com/github/JeffersonEspinalA/Prueba/blob/main/Calculadora_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install flex
!sudo apt install bison

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libfl-dev libfl2
Suggested packages:
  bison flex-doc
The following NEW packages will be installed:
  flex libfl-dev libfl2
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 334 kB of archives.
After this operation, 1,127 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 flex amd64 2.6.4-6.2 [317 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libfl2 amd64 2.6.4-6.2 [11.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 libfl-dev amd64 2.6.4-6.2 [6,316 B]
Fetched 334 kB in 1s (426 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 3.)
debconf: falling back to frontend: Readline
de

In [2]:
%%file calc.l
%{

#include "calc.h"
#include "calc.tab.h"

%}

NUM ([0-9]+\.[0-9]+|([0-9]+\.)|(\.[0-9]+)|([0-9]+))([Ee][+\-]?[0-9]+)?
FUN (atan)|(cos)|(exp)|(ln)|(sin)|(sqrt)
VAR [A-Za-z_][A-Za-z_0-9]*
OP [+\-*/=\^()\n]

%%

{OP}          { return yytext[0]; }
{FUN}         { yylval.VAR = getsym(yytext); return FUN; }
{VAR}         {
	symrec* s = getsym(yytext);
	if (!s) s = putsym(yytext, VAR);
	yylval.VAR = s;
	return VAR; }
{NUM}         { yylval.NUM = atof(yytext); return NUM; }
[ \t]         ;
.             ; /* unknown token */

%%

Writing calc.l


In [3]:
%%file calc.y
%{

/* código C, importar scanner */
#include <stdlib.h>
#include <ctype.h>
#include <stdio.h>
#include <math.h>

#include "calc.h"

int yylex(void);
void yyerror(char const* s) {
	fprintf(stderr, "%s\n", s);
}

%}

%define api.value.type union
%token <double> NUM
%token <symrec*> VAR FUN
%nterm <double> exp

%precedence '='
%left '-' '+'
%left '*' '/'
%precedence NEG
%right '^'

%verbose

%define parse.trace

%printer { fprintf(yyo, "%s", $$->name); } VAR;
%printer { fprintf(yyo, "%s()", $$->name); } FUN;
%printer { fprintf(yyo, "%g", $$); } <double>;

%%

input
	: %empty
	| input line
	;

line
	: '\n'
	| exp '\n'                 { printf("= %f\n", $1); }
	| error '\n'               { yyerrok; }
	;

exp
	: NUM
	| VAR                      {
		if ($1->init)
			$$ = $1->value.var;
		else {
			yyerror("undef var");
			yyerrok;
		}
	}
	| VAR '=' exp              { $$ = $3; $1->value.var = $3; $1->init = 1; }
	| FUN '(' exp ')'          { $$ = $1->value.fun($3); }
	| exp '+' exp              { $$ = $1 + $3; }
	| exp '-' exp              { $$ = $1 - $3; }
	| exp '*' exp              { $$ = $1 * $3; }
	| exp '/' exp              { $$ = $1 / $3; }
	| '-' exp %prec NEG        { $$ = -$2; }
	| exp '^' exp              { $$ = pow($1, $3); }
	| '(' exp ')'              { $$ = $2; }
	;

%%

struct init {
	char const* name;
	func_t* fun;
};

struct init const funs[] = {
	{ "atan", atan },
	{ "cos", cos },
	{ "exp", exp },
	{ "ln", log },
	{ "sin", sin },
	{ "sqrt", sqrt },
	{ 0, 0 },
};

symrec* sym_table;

static void init_table(void) {
	for (int i = 0; funs[i].name; ++i) {
		symrec* ptr = putsym(funs[i].name, FUN);
		ptr->value.fun = funs[i].fun;
		ptr->init = 1;
	}
}

#include <assert.h>
#include <stdlib.h>
#include <string.h>

symrec* putsym(char const* name, int sym_type) {
	symrec* res = (symrec*) malloc(sizeof(symrec));
	res->name = strdup(name);
	res->type = sym_type;
	res->value.var = 0;
	res->init = 0;
	res->next = sym_table;
	sym_table = res;
	return res;
}

symrec* getsym(char const* name) {
	for (symrec* p = sym_table; p; p = p->next) {
		if (strcmp(p->name, name) == 0) {
			return p;
		}
	}
	return NULL;
}

int main(int argc, char const* argv[]) {
	if (argc == 2 && strcmp(argv[1], "-p") == 0) {
		yydebug = 1;
	}
	init_table();
	return yyparse();
}

Writing calc.y


In [4]:
%%file calc.h
#ifndef __CALC_H__
#define __CALC_H__

typedef double (func_t)(double);

/* based on GNU Bison's implementation */
/* linked list node for symbol table */
struct symrec {
	char* name;
	int type;
	union {
		double var;
		func_t *fun;
	} value;
	int init;
	struct symrec* next;
};

typedef struct symrec symrec;

extern symrec* sym_table;

symrec* putsym(char const* name, int sym_type);
symrec* getsym(char const* name);


#endif

Writing calc.h


In [5]:
%%file Makefile
CC := gcc
CP := cp
RM := rm -rf
MD := mkdir -p
YACC := bison
LEX := flex

CCFLAGS += -lfl -lm
YACCFLAGS += -d

all: calc.tab.c lex.yy.c
	$(CC) calc.tab.c lex.yy.c $(CCFLAGS)

run:
	./a.out

calc.tab.c:
	$(YACC) $(YACCFLAGS) calc.y

lex.yy.c:
	$(LEX) calc.l

clean:
	$(RM) a.out
	$(RM) calc.tab.c
	$(RM) calc.tab.h
	$(RM) lex.yy.c
	$(RM) calc.output

Writing Makefile


In [6]:
!make

bison -d calc.y
flex calc.l
gcc calc.tab.c lex.yy.c -lfl -lm


In [7]:
!./a.out

45
= 45.000000
34 54.3 +
syntax error
3 + 2
= 5.000000
^C
